# **Urban Infrastructure Stress Analysis:**

Geospatial Data Analysis – HSLU Master’s Program


## Mount Google Drive in Collab

In [ ]:
from google.colab import drive
from pathlib import Path

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Base path for the project (EDIT THIS ONLY)
BASE = Path("/content/drive/MyDrive/Colab Notebooks/GEO Spatial Data Analysis")

# 3. Data folders
DATA = BASE / "Data"
RAW = DATA / "raw"
PROC = DATA / "processed"

print("Project base folder:", BASE)
print("Raw data folder:", RAW)
print("Processed data folder:", PROC)

# Check if raw folder exists
if RAW.exists():
    print("\nFiles in RAW folder:")
    for p in RAW.iterdir():
        print(" -", p.name)
else:
    print("\n❌ ERROR: RAW folder not found. Please check your BASE path.")


Mounted at /content/drive
Project base folder: /content/drive/MyDrive/Colab Notebooks/GEO Spatial Data Analysis
Raw data folder: /content/drive/MyDrive/Colab Notebooks/GEO Spatial Data Analysis/Data/raw
Processed data folder: /content/drive/MyDrive/Colab Notebooks/GEO Spatial Data Analysis/Data/processed

❌ ERROR: RAW folder not found. Please check your BASE path.


## Importing the necessary libaries

In [11]:
# =========================================================
# 0. IMPORTS
# =========================================================

# --- Core Python Libraries ---
import pandas as pd
import numpy as np

# --- File & Path Handling ---
from pathlib import Path

# --- Geospatial Libraries (added later if needed) ---
import geopandas as gpd
from shapely.geometry import Point
from pyproj import Transformer

# --- Visualization Libraries (if needed in Colab) ---
# import matplotlib.pyplot as plt
# import seaborn as sns

# --- Machine Learning / Clustering (added later) ---
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import StandardScaler

# --- Utility ---
import warnings
warnings.filterwarnings("ignore")

print("Imports loaded successfully.")


Imports loaded successfully.


## Loading and inspecting the raw data

In [ ]:
# =========================================================
# 2. LOAD AND INSPECT RAW DATA
# =========================================================

import pandas as pd

# 2.1 Define file paths (using RAW from Section 1)
traffic_path = RAW / "traffic_data_cleaned.csv"
pop_path     = RAW / "population_raw.csv"
quart_path   = RAW / "str_stadtquartier_raw_for_join.csv"

print("Traffic file path :", traffic_path)
print("Population path   :", pop_path)
print("Quartier path     :", quart_path)

# 2.2 Traffic data (sample only, file is very large)
print("\n=== TRAFFIC DATA (sample) ===")
print("Approx. file size (GB):", round(traffic_path.stat().st_size / 1e9, 3))

traffic_sample = pd.read_csv(traffic_path, nrows=100_000)  # sample for inspection
print("Sample shape:", traffic_sample.shape)
display(traffic_sample.head())
print("\nTraffic columns:\n", traffic_sample.columns.tolist())

# 2.3 Population data (load full)
print("\n=== POPULATION DATA ===")
population = pd.read_csv(pop_path)
print("Population shape:", population.shape)
display(population.head())
print("\nPopulation columns:\n", population.columns.tolist())

# 2.4 Quartier lookup table
print("\n=== QUARTIER LOOKUP TABLE ===")
quartiers = pd.read_csv(quart_path)
print("Quartier table shape:", quartiers.shape)
display(quartiers.head())
print("\nQuartier columns:\n", quartiers.columns.tolist())


### Traffic Data

In [ ]:
# =========================================================
# 3A. TRAFFIC DATA PREPARATION
# =========================================================


# Use only the columns we need
traffic_cols = [
    "measurement_site_id",
    "east_coordinate",
    "north_coordinate",
    "timestamp",
    "vehicle_count"
]

traffic_path = RAW / "traffic_data_cleaned.csv"

# Load only the needed columns
traffic = pd.read_csv(
    traffic_path,
    usecols=traffic_cols,
    parse_dates=["timestamp"]  # auto-parse datetime
)

print("Traffic loaded.")
traffic.head()


| Column                  | Why we need it                                                     |
| ----------------------- | ------------------------------------------------------------------ |
| **measurement_site_id** | Groups all records from the same station; ensures unique locations |
| **east_coordinate**     | Needed to map station location into Zurich quarters                |
| **north_coordinate**    | Needed to map station location into Zurich quarters                                                     |
| **timestamp**           | Extract year (and later day/hour patterns if needed)               |
| **vehicle_count**       | The actual traffic value                                           |


Traffic Data Quality Checks

In [ ]:
# =========================================================
# 3A.4 TRAFFIC DATA QUALITY CHECKS
# =========================================================

print("Checking missing values...\n")
missing = traffic.isna().sum()
print(missing)

print("\nChecking duplicates...\n")
duplicate_rows = traffic.duplicated().sum()
print("Total duplicate rows:", duplicate_rows)

print("\nChecking unique measurement sites...\n")
print("Unique measurement_site_id:", traffic["measurement_site_id"].nunique())

print("\nChecking timestamp range...\n")
print("Min timestamp:", traffic["timestamp"].min())
print("Max timestamp:", traffic["timestamp"].max())

print("\nChecking coordinate sanity...\n")
print("East min/max:", traffic["east_coordinate"].min(), traffic["east_coordinate"].max())
print("North min/max:", traffic["north_coordinate"].min(), traffic["north_coordinate"].max())


Cleaning and saving a new file

In [ ]:
# =========================================================
# 3A.5 CLEAN TRAFFIC DATA (remove missing & duplicates)
# =========================================================

traffic_clean = traffic.copy()

# 1. Remove rows with missing vehicle_count
before_missing = traffic_clean.shape[0]
traffic_clean = traffic_clean.dropna(subset=["vehicle_count"])
after_missing = traffic_clean.shape[0]

print(f"Removed {before_missing - after_missing} rows with missing vehicle_count.")

# 2. Remove duplicate rows
before_dupes = traffic_clean.shape[0]
traffic_clean = traffic_clean.drop_duplicates()
after_dupes = traffic_clean.shape[0]

print(f"Removed {before_dupes - after_dupes} duplicate rows.")

# 3. Reset index
traffic_clean = traffic_clean.reset_index(drop=True)

traffic_clean.info()


In [ ]:
# =========================================================
# 3A.6 CONVERT COORDINATES FROM LV95 TO WGS84
# =========================================================

# Create a transformer from Swiss LV95 (EPSG:2056) to WGS84 (EPSG:4326)
# LV95 uses East/North, WGS84 uses Lat/Lon
transformer = Transformer.from_crs("EPSG:2056", "EPSG:4326", always_xy=True)

print("Converting coordinates from LV95 (EPSG:2056) to WGS84 (EPSG:4326)...")

# Transform coordinates
# transformer.transform expects (x, y) which is (east, north) for LV95
# and returns (longitude, latitude) for WGS84
traffic_clean["longitude"], traffic_clean["latitude"] = transformer.transform(
    traffic_clean["east_coordinate"].values,
    traffic_clean["north_coordinate"].values
)

print("Conversion complete!")
print("\nSample of converted coordinates:")
display(traffic_clean[["measurement_site_id", "east_coordinate", "north_coordinate",
                       "longitude", "latitude"]].head(10))

# Quick sanity check: Zurich coordinates should be around:
# Latitude: ~47.37 (North)
# Longitude: ~8.54 (East)
print("\nCoordinate ranges (should be in Zurich area):")
print(f"Latitude range: {traffic_clean['latitude'].min():.4f} to {traffic_clean['latitude'].max():.4f}")
print(f"Longitude range: {traffic_clean['longitude'].min():.4f} to {traffic_clean['longitude'].max():.4f}")

In [ ]:
clean_path = PROC / "traffic_clean_small.csv"
traffic_clean.to_csv(clean_path, index=False)
print("Saved cleaned file to:", clean_path)


### Population Data

In [ ]:
# 3B.1 Load population dataset

pop_path = RAW / "population_raw.csv"
print("Population file path:", pop_path)

population = pd.read_csv(pop_path)

population.head()
population.columns.tolist()



Loding only the needed Column

In [ ]:
# Look at how categories are coded in the original dataset
population["SexCd"].unique(), population["HerkunftCd"].unique(), population["AlterV20ueber80Cd_noDM"].unique()


In [ ]:
population_jan = population[population["StichtagDatMM"] == 1]
population_jan.head()


In [ ]:
# ============================================================
# Inspect Demographic Categories in the Population Dataset
# This helps us understand how the dataset is split into
# demographic subgroups (sex, origin, age).
# We will use this to correctly aggregate total population.
# ============================================================

# Check how sex is coded (male/female)
print("SEX CODES (SexCd):")
print(population_jan["SexCd"].value_counts())
print("\n1 = male, 2 = female")

# Check how origin (Swiss/Foreign) is coded
print("\nORIGIN CODES (HerkunftCd):")
print(population_jan["HerkunftCd"].value_counts())
print("\n1 = Swiss, 2 = Foreign")

# Check age group codes
print("\nAGE GROUP CODES (AlterV20ueber80Cd_noDM):")
print(population_jan["AlterV20ueber80Cd_noDM"].value_counts())
print("\nAge groups 1–5 represent different age bands (e.g. 0–19, 20–39, etc.)")



NOTE:
Each quarter-year contains multiple demographic rows:
- 2 sexes × 2 origins × 5 age groups = 20 rows per quarter-year.
To obtain TRUE population, we must sum over all demographic categories

In [ ]:
# ============================================================
# 3B.4 Aggregate TRUE population per year & quarter (January only)
# We now:
# 1) Keep only the columns we need for aggregation
# 2) Group by Year + Quarter + Quarter name
# 3) Sum AnzBestWir over ALL demographic categories
#    (both sexes, both origins, all age groups)
#    -> this gives the TRUE population for each quarter-year.
# ============================================================

# 1) Keep only the relevant columns from the January snapshot
population_jan_small = population_jan[[
    "StichtagDatJahr",   # year
    "QuarCd",            # quarter ID (numeric)
    "QuarLang",          # quarter name (string)
    "AnzBestWir"         # population count for each demographic slice
]]

# 2) Aggregate to total population per (year, quarter)
population_year_quarter = (
    population_jan_small
    .groupby(["StichtagDatJahr", "QuarCd", "QuarLang"], as_index=False)
    .agg(total_population=("AnzBestWir", "sum"))
)

# Quick look at the result
population_year_quarter.head()


In [ ]:
population_year_quarter.groupby("StichtagDatJahr")["total_population"].sum()


Sanity Check

In [ ]:
# ============================================================
# 3B.7 Check for Missing Values and Duplicates in Final Dataset
# This is important before saving the clean dataset to disk.
#
# We check:
# - Missing values in any column
# - Total number of duplicated rows
# - Dataset size (rows, columns)
# ============================================================

print("=== CHECK MISSING VALUES ===\n")
print(population_year_quarter.isna().sum())

print("\n=== CHECK DUPLICATED ROWS ===\n")
dup_count = population_year_quarter.duplicated().sum()
print("Number of duplicated rows:", dup_count)

print("\n=== DATASET SHAPE ===")
print("Rows:", population_year_quarter.shape[0])
print("Columns:", population_year_quarter.shape[1])


In [ ]:
# ============================================================
# 3Save Cleaned Population Dataset
#
# This file will be used in:
#  - Tableau visualizations
#  - Spatial join with Zurich quarters
#  - Merging with traffic data
#  - Stress Index calculation
# ============================================================

save_path = PROC / "population_year_quarter_clean.csv"
population_year_quarter.to_csv(save_path, index=False)

print("Clean population dataset saved to:")
print(save_path)


### Quartier Data

In [ ]:
# ============================================================
# 3C.1 Load Quartier Lookup Dataset
# This dataset maps addresses / GWR IDs to:
#  - statistisches_quartier (quarter)
#  - stadtkreis (district)
#  - address information
# ============================================================

quartier_path = RAW / "str_stadtquartier_raw_for_join.csv"

quartiers = pd.read_csv(quartier_path)

print("Quartier dataset loaded.")
print("Shape:", quartiers.shape)
quartiers.head()


Selecting Relevant Columns

In [ ]:
# ================================================
# 3C.2 Keep Only Relevant Quartier Columns
#  - gwr_egid: building ID (sometimes useful for merges)
#  - adresse: optional for labels
#  - stadtkreis: district ID
#  - statistisches_quartier: quarter name
# ================================================

quartier_clean = quartiers[[
    "gwr_egid",
    "adresse",
    "stadtkreis",
    "statistisches_quartier"
]].copy()

quartier_clean.head()


Sanity Check

In [ ]:
# ============================================================
# 3C.3 Sanity Check for Quartier Lookup (clean version)
# Checks:
#   - Missing values per column
#   - Number of duplicated rows
#   - Final dataset shape
# ============================================================

print("=== SANITY CHECK: Quartier Lookup Dataset ===\n")

# 1) Missing values per column
missing = quartier_clean.isna().sum()
print(">> Missing values per column:\n")
print(missing)
print("\n")

# 2) Duplicate rows
duplicates = quartier_clean.duplicated().sum()
print(">> Number of duplicated rows:", duplicates)
print("\n")

# 3) Dataset shape
rows, cols = quartier_clean.shape
print(">> Dataset shape:")
print(f"Rows: {rows}")
print(f"Columns: {cols}")

print("\n=== Sanity Check Completed ===")


In [ ]:
# ============================================================
# 3C.4 Save Clean Quartier Dataset
# This makes the dataset ready for merging and visualization
# ============================================================

# Define save path
SAVE_DIR = "/content/drive/MyDrive/Colab Notebooks/GEO Spatial Data Analysis/Data/processed/"
quartier_save_path = SAVE_DIR + "quartier_clean.csv"

# Save the dataframe
quartier_clean.to_csv(quartier_save_path, index=False)

print("Quartier dataset saved successfully at:")
print(quartier_save_path)


Quartier Shape File

In [ ]:
RAW = "/content/drive/MyDrive/Colab Notebooks/GEO Spatial Data Analysis/Data/raw/"

# See what is inside the raw folder
!ls "$RAW"


In [ ]:
QUARTER_DIR = RAW + "Stadtkreise_und_Quartiere_Zurich_und_Winterthur_-OGD/"

# List all files in that folder
!ls "$QUARTER_DIR"


In [ ]:
import geopandas as gpd

quartier_shp = QUARTER_DIR + "UP_STADTQUARTIERE_F.shp"

quarters = gpd.read_file(quartier_shp)

print("Loaded quarter shapefile!")
print("Shape:", quarters.shape)
print(quarters.columns)

quarters.head()


Quarter Mapping

In [ ]:
import geopandas as gpd

# You already have this:
# quarters = gpd.read_file(quartier_shp)

# Keep only Zurich (not Winterthur)
quarters_zurich = quarters[quarters["GEMEINDENA"] == "Zürich"].copy()
print("Zurich quarters:", quarters_zurich.shape)

# Ensure CRS is LV95
print("Quarters CRS before:", quarters_zurich.crs)
if quarters_zurich.crs is None:
    quarters_zurich = quarters_zurich.set_crs(epsg=2056)


### Loading the Clean Version

In [ ]:
# ==========================================================
# LOAD ALL CLEANED DATASETS FROM /processed FOLDER
# ==========================================================


BASE = "/content/drive/MyDrive/Colab Notebooks/GEO Spatial Data Analysis/Data/processed/"

traffic_clean_path = BASE + "traffic_clean_small.csv"
population_clean_path = BASE + "population_year_quarter_clean.csv"
quartier_clean_path = BASE + "quartier_clean.csv"

# Load them
traffic_clean = pd.read_csv(traffic_clean_path)
population_clean = pd.read_csv(population_clean_path)
quartier_clean = pd.read_csv(quartier_clean_path)

print("Datasets loaded successfully.\n")

print("Traffic shape:", traffic_clean.shape)
print("Population shape:", population_clean.shape)
print("Quartier shape:", quartier_clean.shape)

print("=== TRAFFIC CLEAN (HEAD) ===")
display(traffic_clean.head())

print("\n=== POPULATION CLEAN (HEAD) ===")
display(population_clean.head())

print("\n=== QUARTIER CLEAN (HEAD) ===")
display(quartier_clean.head())


## Merging Dataset

Quartier Mapping

### Quarter Mapping via Spatial Join

Goal: Assign each traffic measurement site to a statistical quarter so that we can later aggregate
traffic volumes per quarter and year.

Inputs:
- `traffic` (cleaned traffic data with `measurement_site_id`, `east_coordinate`, `north_coordinate`)
- `quarters_zurich` (GeoDataFrame with quarter polygons from `UP_STADTQUARTIERE_F.shp`)

Method:
1. Extract all **unique measurement sites** and their coordinates.
2. Convert the sites to a GeoDataFrame in the Swiss coordinate system LV95 (EPSG:2056).

1.   List item
2.   List item


3. Use a **spatial join** (`gpd.sjoin`) between the sites and the quarter polygons.
4. This assigns each site to a quarter name (`QUARTIERNA`) and quarter number (`QUARTIERNR`).

Output:
- `sites_with_quarter`: GeoDataFrame with one row per measurement site and its assigned quarter.


In [ ]:

print("=== Traffic Columns ===")
print(traffic.columns)
print("\nTraffic preview:")
display(traffic.head())

print("\n=== Quartier Columns ===")
print(quartier.columns)
print("\nQuartier preview:")
display(quartier.head())

print("\n=== Population Columns ===")
print(population.columns)
print("\nPopulation preview:")
display(population.head())


In [ ]:
# Keep only needed columns from Zurich quarters
quarters_slim = quarters_zurich[["QUARTIERNA", "QUARTIERNU", "geometry"]]

# Spatial join: assign each site to a quarter
sites_with_quarter = gpd.sjoin(
    gdf_sites,
    quarters_slim,
    how="left",
    predicate="within"
)

print("Sites with quarter info:", sites_with_quarter.shape)
sites_with_quarter.head()


In [ ]:
# Create a clean site → quarter mapping and merge into full traffic data

# 5.1 Make a clean mapping table (drop geometry + index from spatial join)
sites_quarter_map = (
    sites_with_quarter
    .drop(columns=["geometry", "index_right"])
    .rename(columns={"QUARTIERNA": "quarter_name",
                     "QUARTIERNU": "quarter_id"})
)

print("Site → quarter mapping:")
display(sites_quarter_map.head())

# 5.2 Merge the mapping back into the full traffic dataset
traffic_q = traffic.merge(
    sites_quarter_map[["measurement_site_id", "quarter_name", "quarter_id"]],
    on="measurement_site_id",
    how="left"
)

print("Traffic with quarter info:", traffic_q.shape)
display(traffic_q.head())


Saving The Qauartier Data with the Quarter Information

In [ ]:

#  Save the enriched traffic dataset with quarter information
output_path = BASE + "traffic_with_quarters.csv"

traffic_q.to_csv(output_path, index=False)

print("Saved enriched traffic dataset to:", output_path)


## Aggreate Yearly Traffic per Quarter

 **Aggregate Yearly Traffic per Quarter**

Goal: Compute the total annual traffic volume for each statistical quarter.

Input:
- `traffic_q`: enriched traffic dataset  
  (one row per measurement event with `vehicle_count`, `timestamp`, `quarter_name`, `quarter_id`)

Method:
1. Convert `timestamp` to a datetime object and extract the calendar year.
2. Group by `quarter_name`, `quarter_id`, and `year`.
3. Sum `vehicle_count` within each group to obtain `total_traffic`.

Output:
- `traffic_qy`: table with one row per (quarter, year) and the corresponding `total_traffic`.
- This table will later be merged with the population dataset to compute growth rates and the Stress Index.


In [ ]:
# Aggregate yearly traffic per quarter

import pandas as pd

# 7.1 Ensure timestamp is datetime and create a 'year' column
traffic_q["timestamp"] = pd.to_datetime(traffic_q["timestamp"], errors="coerce")

# Drop rows where timestamp could not be parsed (if any)
before_drop = traffic_q.shape[0]
traffic_q = traffic_q.dropna(subset=["timestamp"])
after_drop = traffic_q.shape[0]

print(f"Dropped {before_drop - after_drop} rows with invalid timestamps.")

traffic_q["year"] = traffic_q["timestamp"].dt.year

# 7.2 Aggregate total yearly traffic per quarter
traffic_qy = (
    traffic_q
    .groupby(["quarter_name", "quarter_id", "year"], as_index=False)["vehicle_count"]
    .sum()
    .rename(columns={"vehicle_count": "total_traffic"})
    .sort_values(["quarter_id", "year"])
)

print("Traffic per quarter & year:", traffic_qy.shape)
display(traffic_qy.head(10))


## Merge Yearly Traffic with Yearly Population Data

### Merge Yearly Traffic with Yearly Population Data

Goal:
Combine the aggregated annual traffic per quarter (`traffic_qy`) with the annual population
per quarter (`population`) so that we can compute growth rates and the Stress Index.

Matching keys:
- Quarter name: `traffic_qy.quarter_name` ↔ `population.QuarLang`
- Year: `traffic_qy.year` ↔ `population.StichtagDatJahr`

Output:
A unified dataset (`traffic_pop`) containing:
- quarter_name
- quarter_id
- year
- total_traffic
- total_population


In [ ]:
# Prepare population dataset for merging
population_renamed = population.rename(
    columns={
        "QuarLang": "quarter_name",
        "StichtagDatJahr": "year"
    }
)

# Merge traffic totals with population totals
traffic_pop = traffic_qy.merge(
    population_renamed[["quarter_name", "year", "total_population"]],
    on=["quarter_name", "year"],
    how="left"
)

print("Merged traffic + population dataset:", traffic_pop.shape)
traffic_pop.head(10)


## Compute Year-over-Year Growth Rates

### Compute Year-over-Year Growth Rates

Goal:
Calculate yearly growth for both traffic volume and population in each quarter.

Method:
1. Sort the merged dataset by `quarter_id` and `year`.
2. Use `groupby` + `pct_change()` to compute the percentage change:
   - `traffic_growth_pct = pct_change(total_traffic)`
   - `population_growth_pct = pct_change(total_population)`
3. Multiply by 100 to convert to percentages.

Output:
Two new columns:
- `traffic_growth_pct`
- `population_growth_pct`

These will be used to compute the Stress Index in the next step.


In [ ]:
#  Compute traffic and population growth

# Ensure correct sorting
traffic_pop = traffic_pop.sort_values(["quarter_id", "year"])

# Compute year-over-year growth
traffic_pop["traffic_growth_pct"] = (
    traffic_pop.groupby("quarter_id")["total_traffic"].pct_change() * 100
)

traffic_pop["population_growth_pct"] = (
    traffic_pop.groupby("quarter_id")["total_population"].pct_change() * 100
)

print("Growth columns added:")
traffic_pop.head(10)


## Computing the Stress Index

###  Compute the Stress Index

The Stress Index measures whether traffic growth outpaces population growth:
$$
\text{Stress Index} = \text{traffic growth (\%)} - \text{population growth (\%)}
$$
Interpretation:
- **Positive values:** Traffic grows faster → more external inflow → *Commuter Hub*
- **Negative values:** Population grows faster → streets used more by residents → *Residential Zone*
- **Near zero:** Traffic and population change together → *Balanced*


In [ ]:
# Compute Stress Index
traffic_pop["stress_index"] = (
    traffic_pop["traffic_growth_pct"] - traffic_pop["population_growth_pct"]
)

print("Stress Index added:")
traffic_pop.head(10)


• Stress Index = Traffic Growth (%) – Population Growth (%)

• Positive Stress Index → traffic grows faster than population
  * Example: 2013 in Rathaus
    - Traffic growth: +3.59%
    - Population growth: +0.83%
    - Stress Index: +2.77% → indicates more commuter pressure

• Negative Stress Index → population grows faster or traffic drops
  * Example: 2015 in Rathaus
    - Traffic growth: –37.83%
    - Population growth: +0.82%
    - Stress Index: –38.65% → reflects a major traffic decline (construction or route changes)

• Very large positive Stress Index → strong traffic rebound or sudden inflow
  * Example: 2016 in Rathaus
    - Traffic growth: +57.58%
    - Population growth: –0.78%
    - Stress Index: +58.36% → could be heavy commuter influence or after construction work is done

• Near-zero Stress Index → traffic and population change together
   * Example: 2021 in Rathaus
    - Traffic growth: +2.52%
    - Population growth: +0.86%
    - Stress Index: +1.67% → relatively balanced

• Population growth is stable each year (around +0.6% to +1%)
• Traffic growth is highly volatile due to real-world factors:
  - roadworks
  - closures
  - sensor outages
  - seasonal variation
  - COVID impact (e.g., 2020 shows reduced traffic)

• Stress Index values in the dataset follow realistic patterns and confirm:
  - traffic does not behave like population
  - commuter flows strongly affect certain quarters
  - volatility in traffic explains extreme stress values


## Final Dataset Export for Tableau

In [ ]:
traffic_pop_export = traffic_pop.rename(columns={
    "quarter_name": "quarter",
    "quarter_id": "quarter_id",
    "year": "year",
    "total_traffic": "total_traffic",
    "total_population": "total_population",
    "traffic_growth_pct": "traffic_growth_pct",
    "population_growth_pct": "population_growth_pct",
    "stress_index": "stress_index",
    "category": "category"
})


In [ ]:
export_path = BASE + "stress_index_tableau_ready.csv"

traffic_pop_export.to_csv(export_path, index=False)

print("Exported Tableau dataset to:", export_path)


In [ ]:
df_export = pd.read_csv(BASE + "stress_index_tableau_ready.csv")
df_export.head(20)


### Classify Quarters Based on Stress Index (Optional)

To make the Stress Index easier to interpret and visualize, we classify each quarter–year
combination into three categories:

- Commuter Hub: Stress Index > +5  
- Residential Zone: Stress Index < –5  
- Balanced: Stress Index between –5 and +5  

This classification helps identify which quarters experience mobility pressure from commuters,
which are more residential, and which remain stable. This enhances storytelling and supports
clear thematic mapping in Tableau.

In [ ]:
# Create a classification based on Stress Index

def classify_stress(x):
    if x > 5:
        return "Commuter Hub"
    elif x < -5:
        return "Residential Zone"
    else:
        return "Balanced"

df_export["category"] = df_export["stress_index"].apply(classify_stress)

print("Classification added:")
df_export.head(20)


**Classification shows which quarters are commuter hubs, residential zones, or balanced.**
* Commuter Hubs: quarters where stress index exceeds +5, indicating traffic growth far above population growth.
* Residential Zones: stress index below –5, where population grows faster or traffic drops.
* Balanced Quarters: stress index between –5 and +5, indicating stable mobility patterns.
* This classification improves the thematic map and helps identify pressure points in the city's mobility network.


In [ ]:
# Create a new export dataset including the classification column
traffic_pop_export_classes = traffic_pop.rename(columns={
    "quarter_name": "quarter",
    "quarter_id": "quarter_id",
    "year": "year",
    "total_traffic": "total_traffic",
    "total_population": "total_population",
    "traffic_growth_pct": "traffic_growth_pct",
    "population_growth_pct": "population_growth_pct",
    "stress_index": "stress_index",
    "category": "category"
})

# New file path
export_path_classes = BASE + "stress_index_tableau_with_classes.csv"

# Save to disk
traffic_pop_export_classes.to_csv(export_path_classes, index=False)

print("Exported classified Tableau dataset to:", export_path_classes)
